# Query Data with AWS Data Wrangler

**AWS Data Wrangler** is an open-source Python library that extends the power of the Pandas library to AWS connecting DataFrames and AWS data related services (Amazon Redshift, AWS Glue, Amazon Athena, Amazon EMR, Amazon QuickSight, etc).

* https://github.com/awslabs/aws-data-wrangler
* https://aws-data-wrangler.readthedocs.io

Built on top of other open-source projects like Pandas, Apache Arrow, Boto3, s3fs, SQLAlchemy, Psycopg2 and PyMySQL, it offers abstracted functions to execute usual ETL tasks like load/unload data from Data Lakes, Data Warehouses and Databases.

_Note that AWS Data Wrangler is simply a Python library that uses existing AWS Services.  AWS Data Wrangler is not a separate AWS Service.  You install AWS Data Wrangler through `pip install` as we will see next._

In [1]:
!pip install -q awswrangler==1.2.0

In [2]:
import boto3
import sagemaker

# Get region 
session = boto3.session.Session()
region_name = session.region_name

# Get SageMaker session & default S3 bucket
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

In [3]:
import awswrangler as wr

# Query Parquet from S3 with Push-Down Filters

Read Apache Parquet file(s) from from a received S3 prefix or list of S3 objects paths.

The concept of Dataset goes beyond the simple idea of files and enable more complex features like partitioning and catalog integration (AWS Glue Catalog): 

dataset (bool) – If True read a parquet dataset instead of simple file(s) loading all the related partitions as columns.

In [4]:
path = '{}/amazon-reviews-pds/parquet/'.format(bucket)
df_parquet_results = wr.s3.read_parquet(path,
                                     columns=['star_rating', 'product_category', 'review_body'],
                                     filters=[("product_category", "=", "Digital_Software")],
                                     dataset=True)
df_parquet_results.shape

(102084, 3)

In [5]:
df_parquet_results.head(5)

,star_rating,review_body,product_category
0,4,So far so good,Digital_Software
1,3,Needs a little more work.....,Digital_Software
2,1,Please cancel.,Digital_Software
3,5,Works as Expected!,Digital_Software
4,4,I've had Webroot for a few years. It expired a...,Digital_Software


# Query Parquet from S3 in Chunks

Batching (chunked argument) (Memory Friendly):

Will enable the function to return a Iterable of DataFrames instead of a regular DataFrame.

There are two batching strategies on Wrangler:
* If chunked=True, a new DataFrame will be returned for each file in your path/dataset.
* If chunked=INTEGER, Wrangler will iterate on the data by number of rows equal to the received INTEGER.

P.S. chunked=True if faster and uses less memory while chunked=INTEGER is more precise in number of rows for each Dataframe.

In [6]:
path = '{}/amazon-reviews-pds/parquet/'.format(bucket)
chunk_iter = wr.s3.read_parquet(path,
                                columns=['star_rating', 'product_category', 'review_body'],
                                filters=[("product_category", "=", "Digital_Software")],
                                dataset=True,
                                chunked=True)

In [7]:
print(next(chunk_iter))

       star_rating                                        review_body  \
0                4                                     So far so good   
1                3                      Needs a little more work.....   
2                1                                     Please cancel.   
3                5                                 Works as Expected!   
4                4  I've had Webroot for a few years. It expired a...   
...            ...                                                ...   
69651            5  Works exactly as it supposed to be. Took 5 min...   
69652            4  I have used this product for over 10 years and...   
69653            1  Without question this is the worst software I ...   
69654            5  I bought this d/l version because it was cheap...   
69655            1  I got this because it has sync capabilities to...   

       product_category  
0      Digital_Software  
1      Digital_Software  
2      Digital_Software  
3      Digital_Soft

# Query the Glue Catalog (ie. Hive Metastore)
Get an iterator of tables.

In [8]:
database_name = 'dsoaws'
table_name_tsv = 'amazon_reviews_tsv'

In [9]:
for table in wr.catalog.get_tables(database="dsoaws"):
    print(table['Name'])

amazon_reviews_parquet
amazon_reviews_tsv


# Query from Athena
Execute any SQL query on AWS Athena and return the results as a Pandas DataFrame.  


In [10]:
%%time
df = wr.athena.read_sql_query(
    sql='SELECT * FROM {} LIMIT 5000'.format(table_name_tsv),
    database=database_name
)

CPU times: user 258 ms, sys: 30.8 ms, total: 289 ms
Wall time: 8.26 s


In [11]:
df.head(5)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,17747349,R2EI7QLPK4LF7U,B00U7LCE6A,106182406,CCleaner Free [Download],Digital_Software,4,0,0,N,Y,Four Stars,So far so good,2015-08-31
1,US,10956619,R1W5OMFK1Q3I3O,B00HRJMOM4,162269768,ResumeMaker Professional Deluxe 18,Digital_Software,3,0,0,N,Y,Three Stars,Needs a little more work.....,2015-08-31
2,US,13132245,RPZWSYWRP92GI,B00P31G9PQ,831433899,Amazon Drive Desktop [PC],Digital_Software,1,1,2,N,Y,One Star,Please cancel.,2015-08-31
3,US,35717248,R2WQWM04XHD9US,B00FGDEPDY,991059534,Norton Internet Security 1 User 3 Licenses,Digital_Software,5,0,0,N,Y,Works as Expected!,Works as Expected!,2015-08-31
4,US,17710652,R1WSPK2RA2PDEF,B00FZ0FK0U,574904556,SecureAnywhere Intermet Security Complete 5 De...,Digital_Software,4,1,2,N,Y,Great antivirus. Worthless customer support,I've had Webroot for a few years. It expired a...,2015-08-31


# Query from Athena in Chunks
Retrieving in chunks can help reduce memory requirements.

In [12]:
%%time

chunk_iter = wr.athena.read_sql_query(
    sql='SELECT * FROM {} LIMIT 5000'.format(table_name_tsv),
    database='{}'.format(database_name),
    chunksize=64_000  # 64 KB Chunks
)

CPU times: user 211 ms, sys: 12.6 ms, total: 224 ms
Wall time: 5.37 s


In [13]:
print(next(chunk_iter))

     marketplace customer_id       review_id  product_id product_parent  \
0             US    21269168   RSH1OZ87OYK92  B013PURRZW      603406193   
1             US      133437  R1WFOQ3N9BO65I  B00F4CEHNK      341969535   
2             US    45765011   R3YOOS71KM5M9  B00DNHLFQA      951665344   
3             US      113118  R3R14UATT3OUFU  B004RMK5QG      395682204   
4             US    22151364   RV2W9SGDNQA2C  B00G9BNLQE      640460561   
...          ...         ...             ...         ...            ...   
4995          US    16345989  R235FLDC348LEU  B005LMOB70       79858647   
4996          US    42941330  R2GBGQKU9HYTOY  B00NQIB41C      337630599   
4997          US    35259719   RAR409BCV0JZ0  B00PG8FOSY      801860929   
4998          US     4527838   RANYM2NKV6J1C  B00MI3YT0Y      270401656   
4999          US    35525773  R24N0TE21SRTRT  B00LU2XHAC      962800663   

                                          product_title     product_category  \
0                 M

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();